In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import requests 
import io
import seaborn as sns

import scipy.sparse as sp

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize

import string
from nltk import word_tokenize
from nltk.corpus import stopwords

from bs4 import BeautifulSoup

nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from transformers import BertTokenizer,BertModel, BertConfig
import torch

### User Mapping

In [ ]:
orig_members = pd.read_csv('kad_mrgcn_public/data/ironmarch/raw_files/orig_members.csv')

In [ ]:
sorted_member_id = orig_members['member_id'].sort_values()
member_id_map ={}
start_id = 1
for id in sorted_member_id:
  member_id_map[id] = start_id
  start_id = start_id + 1

### Message Posts

In [ ]:
orig_message_posts = pd.read_csv('kad_mrgcn_public/data/ironmarch/raw_files/orig_message_posts.csv')

In [ ]:
orig_message_posts.drop(orig_message_posts.columns[[0,1, 2, 3, 5, 7, 8]], axis =1, inplace=True)

### Functions

In [ ]:
def preprocess_text(text):

    soup = BeautifulSoup(text, "lxml")
    text = soup.body.text

    # Get a list of sentences, removes "\n"
    lst = sent_tokenize(text)
    text = " ".join(lst)

    text = text.replace("\n", "")

    # Preprocess using NLTK
    text = text.lower()

    # Remove Punctuation
    text_p = "".join([char for char in text if char not in string.punctuation])

    # Get a List of words 
    words = word_tokenize(text_p)

    words = " ".join(words)

    # Remove stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]

    ret = " ".join(filtered_words)

    return ret


In [ ]:
def getBERT(post):

  tokens = tokenizer.encode_plus(post,return_tensors='pt', 
                                 max_length=512, truncation=True)
  
  indexed_tokens = tokens['input_ids']
  tokens_tensor = indexed_tokens.to('cuda')

  with torch.no_grad():
    # See the models docstrings for the detail of the inputs
    outputs = model(tokens_tensor)
    last_hidden_states = outputs.last_hidden_state
  

# # Take the average
  n_tensor = last_hidden_states.shape[1]
  vector = (last_hidden_states.sum(axis=1)/n_tensor)

  return vector

In [ ]:
def get_word_split(text1):
  l_total = []
  l_partial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150 + 1
  else: 
    n = 1
  # print(n)
  for w in range(n):
    if w == 0:
      l_partial = text1.split()[:200]
      l_total.append(" ".join(l_partial))
    else:
      l_partial = text1.split()[w*150:w*150 + 200]
      l_total.append(" ".join(l_partial))
  return l_total

### Variables

In [ ]:
embedding = np.zeros((orig_members.shape[0], 768))
count = np.zeros(orig_members.shape[0])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

### Iterations

In [ ]:
for i, row in orig_message_posts.iterrows():
  author_id = row['msg_author_id']
  if author_id not in member_id_map:
          continue
  id = member_id_map[author_id]
  doc = row['msg_post']

  clean_doc = preprocess_text(doc)
  clean_chunks = get_word_split(clean_doc)

  
  embed_doc = torch.tensor(np.zeros((1, 768)))
  embed_doc = embed_doc.to('cuda')

  for chunkNo, chunk in enumerate(clean_chunks):
    embed_chunk = getBERT(chunk)
    # embed_chunk = embed_chunk.detach().numpy()
    embed_doc = (embed_doc*chunkNo + embed_chunk)/(chunkNo+1)


  embedding[id-1] = (count[id-1]*embedding[id-1] + embed_doc)/(count[id-1]+1)
  count[id-1] = count[id-1]+1

  if i%5000==0:
    num = int(i/5000)
    np.save("kad_mrgcn_public/data/ironmarch/procesed/msg_posts_embedding_head_v{}".format(i+1), embedding)


In [ ]:
np.save("kad_mrgcn_public/data/ironmarch/processed/msg_posts_embedding_head_final".format(i+1), embedding)